In [1]:
import pandas as pd
import numpy as np
import pickle
import os, time
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

In [2]:
# Reading CSV files, and merging all of them into a single DataFrame
CISIDS2017_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CICIDS2017ML"
CTU13_folder = "/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/CTU13ML"

CICIDS2017_df_list = []
CTU13_df_list = []

# Reading CICIDS2017 CSV files into a single DataFrame
for f in os.listdir(CISIDS2017_folder):
    file_path = os.path.join(CISIDS2017_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CICIDS2017_df_list.append(pd.read_csv(file_path))

CICIDS2017_df = pd.concat(CICIDS2017_df_list, ignore_index=True)

# Reading CTU13 CSV files into a single DataFrame
for f in os.listdir(CTU13_folder):
    file_path = os.path.join(CTU13_folder, f)
    if os.path.isfile(file_path):
        print("Reading: ", f)
        CTU13_df_list.append(pd.read_csv(file_path))

CTU13_df = pd.concat(CTU13_df_list, ignore_index=True)

Reading:  Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Morning.pcap_ISCX_Relabeled.csv
Reading:  Wednesday-workingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX_Relabeled.csv
Reading:  Monday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Friday-WorkingHours-Afternoon-DDos.pcap_ISCX_Relabeled.csv
Reading:  Tuesday-WorkingHours.pcap_ISCX_Relabeled.csv
Reading:  Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX_Relabeled.csv
Reading:  CTU13_Normal_Traffic_Relabeled.csv
Reading:  CTU13_Attack_Traffic_Relabeled.csv


In [3]:
# QUICK PREPROCESSING. 
# Some classifiers do not like "infinite" (inf) or "null" (NaN) values.
CICIDS2017_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CICIDS2017_df.columns[CICIDS2017_df.isna().any()]))
CICIDS2017_df.dropna(inplace=True)

CTU13_df.replace([np.inf, -np.inf], np.nan, inplace=True)
print("Columns with problematic values: ", list(CTU13_df.columns[CTU13_df.isna().any()]))
CTU13_df.dropna(inplace=True)

Columns with problematic values:  [' Flow Packets/s']
Columns with problematic values:  []


In [4]:
CICIDS2017_df[' Label'].unique()

array(['BENIGN', 'PortScan', 'Bot', 'DoS slowloris', 'DoS Slowhttptest',
       'DoS Hulk', 'DoS GoldenEye', 'Heartbleed',
       'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection', 'DDoS', 'FTP-Patator', 'SSH-Patator',
       'Infiltration'], dtype=object)

In [5]:
CTU13_df[' Label'].unique()

array(['BENIGN', 'Bot'], dtype=object)

In [6]:
# Create a new column that unifies all malicious classes into a single class for binary classification
CICIDS2017_df['GT'] = np.where(CICIDS2017_df[' Label']=='BENIGN', 'Benign', 'Malicious')
CTU13_df['GT'] = np.where(CTU13_df[' Label']=='BENIGN', 'Benign', 'Malicious')

In [7]:
features = pd.Index([
    ' Flow Duration',
    ' Total Fwd Packets',
    ' Total Backward Packets',
    ' Total Length of Bwd Packets',
    ' Fwd Packet Length Max',
    ' Fwd Packet Length Min',
    ' Fwd Packet Length Mean',
    ' Fwd Packet Length Std',
    ' Bwd Packet Length Min',
    ' Bwd Packet Length Mean',
    ' Bwd Packet Length Std',
    ' Flow Packets/s',
    ' Flow IAT Mean',
    ' Flow IAT Std',
    ' Flow IAT Max',
    ' Flow IAT Min',
    ' Fwd IAT Mean',
    ' Fwd IAT Std',
    ' Fwd IAT Max',
    ' Fwd IAT Min',
    ' Bwd IAT Mean',
    ' Bwd IAT Std',
    ' Bwd IAT Max',
    ' Bwd IAT Min',
    ' Bwd PSH Flags',
    ' Fwd Header Length',
    ' Bwd Header Length',
    ' Bwd Packets/s',
    ' Min Packet Length',
    ' Max Packet Length',
    ' Packet Length Mean',
    ' Packet Length Std',
    ' Packet Length Variance',
    ' SYN Flag Count',
    ' RST Flag Count',
    ' ACK Flag Count',
    ' Down/Up Ratio',
    ' Average Packet Size',
    ' Avg Fwd Segment Size',
    ' Avg Bwd Segment Size',
    ' Init_Win_bytes_backward',
    ' act_data_pkt_fwd',
    ' Active Std',
    ' Active Max',
    ' Active Min',
    ' Idle Std',
    ' Idle Max',
    ' Idle Min'
])

In [8]:
# Binary SVM Classifier trained and tested on CTU13 dataset
train_ctu13, test_ctu13 = train_test_split(CTU13_df, test_size=0.4, stratify=CTU13_df['GT'])
train_ctu13, val_ctu13 = train_test_split(train_ctu13, test_size=0.25, stratify=train_ctu13['GT'])

start = time.time()
svmClf_bin_ctu13 = LinearSVC()
svmClf_bin_ctu13.fit(train_ctu13[features], train_ctu13['GT'])
end = time.time() - start
print("Training time (CTU13 Binary): ", end)

predictions_bin_ctu13 = svmClf_bin_ctu13.predict(test_ctu13[features])
print("Acc (CTU13 Binary): {:3f}".format(accuracy_score(test_ctu13['GT'], predictions_bin_ctu13)))
print("F1-score (CTU13 Binary): {:3f}".format(f1_score(test_ctu13['GT'], predictions_bin_ctu13, pos_label='Malicious')))
pd.crosstab(test_ctu13['GT'], predictions_bin_ctu13, rownames=['True'], colnames=['Pred'])

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training time (CTU13 Binary):  7.656667232513428
Acc (CTU13 Binary): 0.814884
F1-score (CTU13 Binary): 0.799306


Pred,Benign,Malicious
True,,
Benign,16460,4866
Malicious,1962,13597


In [9]:
# Binary SVM Classifier trained and tested on CICIDS2017 dataset
# Randomly discard 90% of the data and train on the remaining 10%
CICIDS2017_df_subset = CICIDS2017_df.sample(frac=0.5, random_state=42)

train_cicids, test_cicids = train_test_split(CICIDS2017_df_subset, test_size=0.4, stratify=CICIDS2017_df_subset['GT'])
train_cicids, val_cicids = train_test_split(train_cicids, test_size=0.25, stratify=train_cicids['GT'])

start = time.time()
svmClf_bin_cicids = LinearSVC()
svmClf_bin_cicids.fit(train_cicids[features], train_cicids['GT'])
end = time.time() - start
print("Training time (CICIDS2017 Binary): ", end)

predictions_bin_cicids = svmClf_bin_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Binary): {:3f}".format(accuracy_score(test_cicids['GT'], predictions_bin_cicids)))
print("F1-score (CICIDS2017 Binary): {:3f}".format(f1_score(test_cicids['GT'], predictions_bin_cicids, pos_label='Malicious')))
pd.crosstab(test_cicids['GT'], predictions_bin_cicids, rownames=['True'], colnames=['Pred'])

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training time (CICIDS2017 Binary):  220.207581281662
Acc (CICIDS2017 Binary): 0.774821
F1-score (CICIDS2017 Binary): 0.231026


Pred,Benign,Malicious
True,,
Benign,419089,35168
Malicious,92188,19131


In [10]:
# Multiclass SVM Classifier trained and tested on CICIDS2017 dataset
# Randomly discard 90% of the data and train on the remaining 10%
CICIDS2017_df_subset = CICIDS2017_df.sample(frac=0.5, random_state=42)

train_cicids, test_cicids = train_test_split(CICIDS2017_df_subset, test_size=0.4, stratify=CICIDS2017_df_subset[' Label'])
train_cicids, val_cicids = train_test_split(train_cicids, test_size=0.25, stratify=train_cicids[' Label'])

start = time.time()
svmClf_multi_cicids = LinearSVC()
svmClf_multi_cicids.fit(train_cicids[features], train_cicids[' Label'])
end = time.time() - start
print("Training time (CICIDS2017 Multiclass): ", end)

predictions_multi_cicids = svmClf_multi_cicids.predict(test_cicids[features])
print("Acc (CICIDS2017 Multiclass): {:3f}".format(accuracy_score(test_cicids[' Label'], predictions_multi_cicids)))
print("F1-score (CICIDS2017 Multiclass): {:3f}".format(f1_score(test_cicids[' Label'], predictions_multi_cicids, average='macro')))
pd.crosstab(test_cicids[' Label'], predictions_multi_cicids, rownames=['True'], colnames=['Pred'])

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training time (CICIDS2017 Multiclass):  13484.734790802002
Acc (CICIDS2017 Multiclass): 0.804313
F1-score (CICIDS2017 Multiclass): 0.135484


Pred,BENIGN,Bot,DDoS,DoS GoldenEye,DoS Hulk,DoS Slowhttptest,DoS slowloris,Infiltration,PortScan,SSH-Patator,Web Attack � Brute Force,Web Attack � Sql Injection,Web Attack � XSS
True,,,,,,,,,,,,,
BENIGN,428182,70,2445,130,10893,4720,464,11,6986,50,44,5,258
Bot,265,0,0,0,98,0,0,0,21,0,0,0,0
DDoS,9940,0,2023,0,11193,2424,0,0,1,0,0,0,0
DoS GoldenEye,1985,11,0,0,64,2,0,0,0,0,0,0,0
DoS Hulk,39828,5,131,1,5632,361,0,0,1,0,0,0,0
DoS Slowhttptest,363,0,1,0,118,598,4,0,0,0,0,0,0
DoS slowloris,561,0,0,0,249,328,14,0,1,1,0,0,0
FTP-Patator,1608,0,0,0,0,0,0,0,1,0,0,0,0
Heartbleed,1,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# set train variable to the full CTU13 dataset
train = CTU13_df

# set test variable to the CICIDS2017 dataset subset
test = CICIDS2017_df_subset

In [12]:
# Train and test a (binary) SVM Classifier, printing some basic performance scores, training time, and confusion matrix
start = time.time()
svmClf_bin = LinearSVC()
svmClf_bin.fit(train[features], train['GT'])
end = time.time() - start
print("Training time: ", end)

predictions_bin = svmClf_bin.predict(test[features])
print("Acc: {:3f}".format(accuracy_score(test['GT'], predictions_bin)))
print("F1-score: {:3f}".format(f1_score(test['GT'], predictions_bin, pos_label = 'Malicious')))
pd.crosstab(test['GT'], predictions_bin, rownames=['True'], colnames=['Pred'])

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training time:  21.267386436462402
Acc: 0.432963
F1-score: 0.316341


Pred,Benign,Malicious
True,,
Benign,426690,708952
Malicious,92803,185493


In [13]:
# Train and test a (multiclass) SVM Classifier, printing some basic performance scores, training time, and confusion matrix
start = time.time()
svmClf_multi = LinearSVC()
svmClf_multi.fit(train[features], train[' Label'])
end = time.time() - start
print("Training time: ", end)

predictions_multi = svmClf_multi.predict(test[features])
print("Acc: {:3f}".format(accuracy_score(test[' Label'], predictions_multi)))
print("F1-score: {:3f}".format(f1_score(test[' Label'], predictions_multi, average='macro')))
pd.crosstab(test[' Label'], predictions_multi, rownames=['True'], colnames=['Pred'])

/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/grassfed37/6CCS3PRJ/dummy-ML_NIDS/.venv/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Training time:  19.68605375289917
Acc: 0.174914
F1-score: 0.023379


Pred,BENIGN,Bot
True,,
BENIGN,246359,889283
Bot,2,959
DDoS,3,63948
DoS GoldenEye,59,5095
DoS Hulk,26917,87980
DoS Slowhttptest,2,2708
DoS slowloris,42,2843
FTP-Patator,771,3252
Heartbleed,0,3
